In [ ]:
import pandas as pd
import numpy as np
import wrds
from pandas.tseries.offsets import *

# import wrds
conn = wrds.Connection(wrds_username='YOUR_ID_HERE')

# LOADING CRSP DATA FROM WRDS --  Bonds data
print('Load Bond Data')
bonds = conn.raw_sql("""
                      select kycrspid, mcaldt, tmretnua, tmtotout
                      from crspq.tfz_mth
                      """)
bonds['mcaldt']   = pd.DataFrame(bonds[['mcaldt']].values.astype('datetime64[ns]')) + MonthEnd(0)
bonds = bonds.rename(columns={"mcaldt":"date","tmretnua":"ret","tmtotout":"me","kycrspid":"idCRSP"}).copy()

# LOADING CRSP DATA FROM WRDS -- T-Bill data
print('Load T-Bill Data')
rf = conn.raw_sql("""
                      select caldt, t30ret, t90ret
                      from crspq.mcti
                      """)
rf['caldt']   = pd.DataFrame(rf[['caldt']].values.astype('datetime64[ns]')) + MonthEnd(0)
rf = rf.rename(columns={"caldt":"date","t30ret":"rf30","t90ret":"rf90"}).copy()

# LOADING CRSP DATA FROM WRDS -- Equity Returns
print('Load Equity Returns Data')
crsp_raw = conn.raw_sql("""
                      select a.permno, a.permco, a.date, b.shrcd, b.exchcd,
                      a.ret, a.retx, a.shrout, a.prc, a.cfacshr
                      from crspq.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/1900' and '12/31/2025'
                      """)
crsp_raw[['permno', 'permco']] = crsp_raw[['permno', 'permco']].astype(int)
crsp_raw['date']   = pd.to_datetime(crsp_raw['date'],format='%Y-%m-%d',errors='ignore')
crsp_raw['date']   = pd.DataFrame(crsp_raw[['date']].values.astype('datetime64[ns]')) + MonthEnd(0)

# LOADING CRSP DATA FROM WRDS -- Deilsting Returns
print('Load Delisting Returns Data')
dlret_raw = conn.raw_sql("""
                     select permno, dlret, dlstdt, dlstcd
                     from crspq.msedelist
                     """)
dlret_raw.permno    = dlret_raw.permno.astype(int)
dlret_raw['dlstdt'] = pd.to_datetime(dlret_raw['dlstdt'])
dlret_raw['date']  = dlret_raw['dlstdt']+MonthEnd(0)

# LOADING CRSP DATA FROM WRDS -- CRSP VW market index (for comparison purposes)
print('Load VW CRSP market index Data')
mkt_csrp = conn.raw_sql("""
                      select date, VWRETD, totval
                      from crspq.msi
                      """)
mkt_csrp['date']   = pd.DataFrame(mkt_csrp[['date']].values.astype('datetime64[ns]')) + MonthEnd(0)
mkt_csrp = mkt_csrp.rename(columns={"vwretd":"mkt_crsp","totval":"mkt_crsp_mktcap"}).copy()

# Save data
# mkt_csrp.to_pickle(data_folder+'mkt_csrp_PS2.pkl')
# crsp_raw.to_pickle(data_folder+'crsp_raw_PS2.pkl')
# dlret_raw.to_pickle(data_folder+'dlret_raw_PS2.pkl')
# rf.to_pickle(data_folder+'rf_PS2.pkl')
# bonds.to_pickle(data_folder+'bonds_PS2.pkl')

# Close connection to WRDS
conn.close()


Loading library list...
Done
Load Bond Data
Load T-Bill Data
Load Equity Returns Data
